# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/ligue-des-champions-feminine-paris-fc-oh-leuven-debuts-historiques-pour-les-louvanistes-direct-video-21h-11611931',
  'Champions League féminine\xa0: OH Louvain remonte deux buts au Paris FC pour des débuts réussis dans la compétition'],
 ['https://www.rtbf.be/article/derriere-les-discours-rassurants-l-europe-inquiete-de-la-situation-politique-en-france-11612383',
  'Derrière les discours rassurants, l’Europe inquiète de la situation politique en France'],
 ['https://www.rtbf.be/article/matias-fernandez-pardo-decline-la-convocation-de-l-espagne-pour-privilegier-la-belgique-11612592',
  'Matias Fernandez-Pardo décline la convocation de l’Espagne, pour privilégier la Belgique\xa0?'],
 ['https://www.rtbf.be/article/direct-guerre-au-proche-orient-minute-de-silence-des-familles-des-victimes-du-7-octobre-sur-le-site-du-festival-nova-11612087',
  'Direct – Guerre au Proche-Orient\xa0: émotion et recueillement à Tel-Aviv pour le deuxième anniversaire du 7-Octobre

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/ligue-des-champion...,Champions League féminine : OH Louvain remonte...
1,https://www.rtbf.be/article/derriere-les-disco...,"Derrière les discours rassurants, l’Europe inq..."
2,https://www.rtbf.be/article/matias-fernandez-p...,Matias Fernandez-Pardo décline la convocation ...
3,https://www.rtbf.be/article/direct-guerre-au-p...,Direct – Guerre au Proche-Orient : émotion et ...
4,https://www.rtbf.be/article/novak-djokovic-rej...,Novak Djokovic rejoint Zizou Bergs en quarts d...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Les Niouzz
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Demain nous appartient
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Tsaatan de Mongolie : les derniers hommes libres
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Tarmac Comedy


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

Champions League féminine : OH Louvain remonte deux buts au Paris FC pour des débuts réussis dans la compétition
A la hauteur du rendez-vous, les Louvanistes ont sans doute payé un peu d’inexpérience et une sacrée dose d’émotion lors d’une entame de partie particulièrement mal gérée. Le Paris FC, vainqueur de la dernière Coupe de France et 3e du dernier championnat français, a en effet fait mouche dès la 3e minute sur une reprise timide mais efficace de  Daphné Corboz.
Louvain a tenté de réagir immédiatement mais s’est à nouveau fait surprendre sur une perte de balle à la 23e minute. La capitaine  Clara Matéo  inscrivant alors le 2-0.
Les débats se sont davantage équilibrés dans la foulée avec plusieurs actions intéressantes de la part des Louvanistes. Constant une différence de niveau moins marquée qu’attendu, OHL a pris son courage à deux mains en deuxième période pour renverser la vapeur.
La montée au jeu de la jeune  Kadhiya De Ceuster , 16 ans, dès le début de la deuxième période 

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


A la hauteur du rendez-vous, les Louvanistes ont sans doute payé un peu d’inexpérience et une sacrée dose d’émotion lors d’une entame de partie particulièrement mal gérée. Le Paris FC, vainqueur de la dernière Coupe de France et 3e du dernier championnat français, a en effet fait mouche dès la 3e minute sur une reprise timide mais efficace de  Daphné Corboz.
Louvain a tenté de réagir immédiatement mais s’est à nouveau fait surprendre sur une perte de balle à la 23e minute. La capitaine  Clara Matéo  inscrivant alors le 2-0.
Les débats se sont davantage équilibrés dans la foulée avec plusieurs actions intéressantes de la part des Louvanistes. Constant une différence de niveau moins marquée qu’attendu, OHL a pris son courage à deux mains en deuxième période pour renverser la vapeur.
La montée au jeu de la jeune  Kadhiya De Ceuster , 16 ans, dès le début de la deuxième période a tout de suite produit l’effet escompté.  Kim Everaerts  a profité de cette nouvelle dynamique pour réduire l’éc

### Création d'un fichier avec le contenu de l'article


In [9]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp